In [118]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import trange
import tqdm
import torchvision
import boto3
import io
import tempfile
import PIL
import time
from datetime import timedelta
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn import model_selection, metrics
import matplotlib.pyplot as plt
import itertools
import timm

In [119]:
# Make sure you're using cuda (GPU) by checking the hardware accelerator under Runtime -> Change runtime type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using:", device)

We're using: cuda


In [120]:
class MyDatasetTest(Dataset):

    def __init__(self, df, transform=None):
        self.image_path = '/kaggle/input/cassava-leaf-disease-classification/test_images'
        self.labels = df
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = self.image_path + '/' + self.labels.iloc[idx]['image_id']

        # Read the image from the file path
        #image = Image.open(img_name)
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)       
        # Transform the image using self.transform
        if self.transform:
            image = self.transform(image=image)["image"]
        return image

In [121]:
# Credit: https://www.kaggle.com/code/aliabdin1/calculate-mean-std-of-images/notebook
mean = np.array([0.42984136, 0.49624753, 0.3129598 ])
std = np.array([0.21417203, 0.21910103, 0.19542212])

In [122]:
albu_transform_test = A.Compose(
        [A.Resize(height=384, width=384),
         A.Normalize(mean = mean, std = std),
         ToTensorV2()
        ])

In [123]:
test = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')

test_dataset = MyDatasetTest(test, transform=albu_transform_test)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, 
                         num_workers=2, pin_memory=True) 

In [124]:
class ResNext(nn.Module):
  def __init__(self, num_classes):
    super(ResNext, self).__init__()
    self.model = timm.create_model('resnext50_32x4d', pretrained=False)#torchvision.models.resnet34(weights='IMAGENET1K_V1')
    self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

  def forward(self, input):
    out = self.model(input)
    return out #F.softmax(out, dim=1)

In [125]:
model_ResNext = ResNext(5).to(device)
model_ResNext.load_state_dict(torch.load('/kaggle/input/cassavaweights/resnext50_tuning_epoch_15_acc_87.6947.pth'))

<All keys matched successfully>

In [126]:
class effnet(nn.Module):
  def __init__(self, num_classes):
    super(effnet, self).__init__()
    self.model = timm.create_model('tf_efficientnet_b3_ns', pretrained=False)
    self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)

  def forward(self, input):
    out = self.model(input)
    return out #F.softmax(out, dim=1)

In [127]:
model_effnet = effnet(5).to(device)
model_effnet.load_state_dict(torch.load('/kaggle/input/cassavaweights/effnetb3ns_acc_87.94.pth'))

<All keys matched successfully>

In [128]:
class vit(nn.Module):
  def __init__(self, num_classes):
    super(vit, self).__init__()
    self.model = timm.create_model('vit_base_patch16_384', pretrained=False)
    self.model.head = nn.Linear(self.model.head.in_features, 5)

  def forward(self, input):
    out = self.model(input)
    return out #F.softmax(out, dim=1)

In [129]:
model_vit = vit(5).to(device)
model_vit.load_state_dict(torch.load('/kaggle/input/cassavaweights/vit_tuning_epoch_3_acc_88.1776.pth'))

<All keys matched successfully>

In [130]:
def inference(test_loader):
    probs = []
    for i, (images) in enumerate(test_loader):
        images = images.to(device)
        avg_preds = []
        model_ResNext.eval()
        with torch.no_grad():
            y_preds_resnext = model_ResNext(images)
        model_effnet.eval()
        with torch.no_grad():
            y_preds_effnet = model_effnet(images)
        model_vit.eval()
        with torch.no_grad():
            y_preds_vit = model_vit(images)
        y_preds = y_preds_resnext*(1/1.39+ 1/1.39+1/0.84+1/3)+y_preds_effnet*(1/1.46+1/1.21+1/0.92+1/3)+y_preds_vit*(1/2.23+1/1.26+1/1.01+1/3)
        y_preds = y_preds.softmax(1)
        avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

In [131]:
predictions = inference(test_loader)
test['label'] = predictions.argmax(1)
test[['image_id', 'label']].to_csv('submission.csv', index=False)
test.head()

,image_id,label
0,2216849948.jpg,4
